In [1]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
import streamlit as st

# Title of the app
st.title("My Streamlit App")

# Add your code here
# For example, if you have a function from Colab:
def my_function(input_data):
    # Your processing logic
    return processed_data

# Streamlit input
user_input = st.text_input("Enter some input:")

if st.button("Submit"):
    result = my_function(user_input)
    st.write("Result:", result)

In [7]:
#import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.cluster import DBSCAN

In [8]:
np.random.seed(42)
tf.random.set_seed(42)

In [9]:
data = {
    'product_id': [1,2,3,4,5,6,7,8],
    'product_name': [
        "Mi Redmi Note 11","Samsung Galaxy M32","Iphone 15","Iphone 15 pro", "HP Pavilion Laptop","Men's Kurta","Women's Saree","Dell Inspiron"
    ],
    'category': [
        "smartphone","smartphone","smartphone","smartphone","laptop","ethnic","ethnic","laptop"
    ],
    'price': [15000,18000,70000,90000,85000,1200,1500,45000],
    'rating': [4.5,4.4,4.8,4.9,4.3,4.6,4.7,4.4]
}
products_df = pd.DataFrame(data)

In [10]:
texts = [
    "hi", "hello", "hey", "namaste", "Radhe Radhe",
    "I need a smartphone", "show me mobile phones",
    "looking for a laptop", "need a notebook",
    "recommend a kurta", "find me a saree", "suggest something for shopping"
]
labels = [
    "greeting", "greeting", "greeting", "greeting", "greeting",
    "search_product", "search_product", "search_product",
    "search_product", "search_product", "search_product", "search_product"
]

In [11]:
label_map = {"greeting": 0, "search_product": 1}
y = np.array([label_map[label] for label in labels])

In [12]:
tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding='post')
max_length = padded_sequences.shape[1]  # Use length of longest sequence

In [13]:
vocab_size = 100
embedding_dim = 16

In [14]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    LSTM(16),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # Two classes: greeting and search_product
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(padded_sequences, y, epochs=100, verbose=0)
print("Intent classification model trained.\n")

Intent classification model trained.



In [17]:
def predict_intent(user_input):
    seq = tokenizer.texts_to_sequences([user_input.lower()])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    pred = model.predict(padded, verbose=0)
    intent_idx = np.argmax(pred)
    inv_label_map = {v: k for k, v in label_map.items()}
    return inv_label_map[intent_idx]

In [18]:
def recommend_product(user_input):
    # Map keywords to categories:
    category_keywords = {
        "smartphone": "smartphone",
        "mobile": "smartphone",
        "phone": "smartphone",
        "laptop": "laptop",
        "notebook": "laptop",
        "kurta": "ethnic",
        "saree": "ethnic",
        "ethnic": "ethnic",
        "traditional": "ethnic"
    }
    found_category = None
    for keyword, category in category_keywords.items():
        if keyword in user_input.lower():
            found_category = category
            break

    if found_category:
        filtered_products = products_df[products_df['category'] == found_category]
        if not filtered_products.empty:
            recommended = filtered_products.sort_values(by='rating', ascending=False).iloc[0]
            return (f"I recommend '{recommended['product_name']}' for you, "
                    f"priced at ₹{recommended['price']} with a rating of {recommended['rating']}.")
    return "Sorry, I couldn't find a product matching your request."


In [19]:
def cluster_products():
    X = products_df[['price', 'rating']].values
    dbscan = DBSCAN(eps=20000, min_samples=1)
    clusters = dbscan.fit_predict(X)
    products_df['cluster'] = clusters
    return products_df[['product_id', 'product_name', 'category', 'price', 'rating', 'cluster']]

In [20]:
print("Product clusters (using DBSCAN):")
print(cluster_products())
print("\n")

Product clusters (using DBSCAN):
   product_id        product_name    category  price  rating  cluster
0           1    Mi Redmi Note 11  smartphone  15000     4.5        0
1           2  Samsung Galaxy M32  smartphone  18000     4.4        0
2           3           Iphone 15  smartphone  70000     4.8        1
3           4       Iphone 15 pro  smartphone  90000     4.9        1
4           5  HP Pavilion Laptop      laptop  85000     4.3        1
5           6         Men's Kurta      ethnic   1200     4.6        0
6           7       Women's Saree      ethnic   1500     4.7        0
7           8       Dell Inspiron      laptop  45000     4.4        2




In [ ]:
print("Welcome to the Indian Shopping Assistant Chatbot!")
print("Type 'quit' to exit.")
while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Thanks for visiting. Dhanyavaad and goodbye!")
        break

    # Determine the intent based on user input
    intent = predict_intent(user_input)

    if intent == "greeting":
        print("Chatbot: Namaste! How can I help you with shopping today?")
    elif intent == "search_product":
        response = recommend_product(user_input)
        print("Chatbot:", response)
    else:
        print("Chatbot: I'm not sure how to help with that. Could you please clarify?")

In [22]:
%%writefile app.py

Overwriting app.py


In [23]:
!streamlit run shopping_chatbot.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: shopping_chatbot.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.126.119.201:8501

⠙⠹⠸⠼Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://true-bars-kiss.loca.lt
